In [1]:
import requests
from transformers.agents.llm_engine import HfApiEngine, MessageRole, get_clean_message_list

openai_role_conversions = {  # Keep this for message cleaning
    MessageRole.TOOL_RESPONSE: "user",
}

class OllamaHfApiWrapper(HfApiEngine):
    def __init__(self, model_name, temperature, base_url="http://localhost:11434/"):
        super().__init__(model_name) 
        self.base_url = base_url
        self.model_name = model_name
        self.temperature = temperature

    def query(self, payload):
        messages = payload.get("inputs", [])
        messages = get_clean_message_list(messages, role_conversions=openai_role_conversions)
        data = {
            "model": self.model_name,
            "messages": messages,
            "stream": False, # Set to false for non-streaming responses
            "options": {"temperature": self.temperature},
        }

        headers = {"Content-Type": "application/json"}

        try:
            response = requests.post(self.base_url + "/api/chat", json=data, headers=headers, timeout=120)

            response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
            response_json = response.json()
            ## llm_output = response_json["choices"][0]["message"]["content"].strip()
            llm_output = response_json["message"]["content"].strip()
            return [{"generated_text": llm_output}] # HfApiEngine expects a list of dictionaries

        except requests.exceptions.RequestException as e:
            print(f"Error during Ollama request: {e}")
            return [{"generated_text": ""}]  # Return empty string in case of error
        except (KeyError, IndexError) as e:
            print(f"Error parsing Ollama response: {e}. Raw response: {response.text}")
            return [{"generated_text": ""}]

# Example usage:
ollama_engine = OllamaHfApiWrapper(model_name="phi3:medium", temperature=0, base_url="http://localhost:11434/")

messages = [{"role": "user", "content": "Write a short poem about the moon."}]

response = ollama_engine.query({"inputs": messages})
print(response)

code = """py
list=[0, 1, 2]

for i in range(4):
    print(list(i))
"""

messages = [{"role": "user", "content": "I have some code that creates a bug: please debug it and return the final code with static range specification, code: {}".format(code)}]

response = ollama_engine.query({"inputs": messages})
print(response)

/home/sercan/anaconda3/envs/general/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to load tokenizer for model phi3:medium: Incorrect path_or_model_id: 'phi3:medium'. Please provide either the path to a local folder or the repo_id of a model on the Hub.. Loading default tokenizer instead.


[{'generated_text': "In night's embrace, she softly glows,\nA silver orb that ebbs and flows.\nShe whispers tales of ancient lore,\nAnd bathes the world in gentle light.\n\nHer craters hold secrets deep,\nOf cosmic dances and celestial sweep.\nIn her eternal cycle she spins,\nGuiding sailors through darkened seas.\n\nShe waxes full with graceful poise,\nA beacon for the dreaming toys.\nHer crescent smile brings hope anew,\nAs stars twinkle in admiration too.\n\nThe moon, our constant friend above,\nIn her celestial dance we find love.\nFor though she wanes and fades away,\nShe'll rise again with dawn of day."}]
[{'generated_text': "Here's the corrected version of your code. The issue was that you were trying to call `list()` on an integer value which is not allowed. Instead, we should use indexing with square brackets []:\n\n```python\nmy_list = [0, 1, e]\n\nfor i in range(4):\n    print(my_list[i])\n```\n\nThis code will now correctly iterate over the list and print each element."}]
